In [ ]:
pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 38.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)


Mounted at /content/drive/


In [ ]:
import os
import librosa
from tqdm import tqdm
from glob import glob
import pandas as pd
import numpy as np

In [ ]:
from jiwer import wer, cer

In [ ]:
habiganj_main = pd.read_excel('/content/drive/MyDrive/Habiganj_Audio/Habiganj_Data.xlsx')
habiganj_main

,Unnamed: 0,Original,External_ID,Annotator,Contents,District,Time
0,1,rec_01.wav,rec_01_audio_0.wav,shah89@student.sust.edu,আইলাম তোমরার সাতে মাতবার লাইগা। বন্দ খর। সুফিয়...,Habiganj,524
1,2,rec_01.wav,rec_01_audio_1.wav,shah89@student.sust.edu,"না এরা দিতো না, এরা হুজুর মানুষ। কেরে দিতো না?...",Habiganj,295
2,3,rec_01.wav,rec_01_audio_2.wav,shah89@student.sust.edu,<> ছেলে সামলানি লাগে। তে ফুলা কিতা অখনোই সামলা...,Habiganj,207
3,4,rec_01.wav,rec_01_audio_3.wav,shah89@student.sust.edu,"ইখানো তো মনো খর খাজ, অন্য খাজ তো খরন লাগে না, ...",Habiganj,668
4,5,rec_01.wav,rec_01_audio_4.wav,shah89@student.sust.edu,"বাফের বাইত আইলে রিল্যাক্স, যেবালা ইচ্ছা ইবালা ...",Habiganj,149
...,...,...,...,...,...,...,...
1179,1180,rec_34.wav,rec_34_audio_17.wav,sabrinayesminreya@gmail.com,টিক আছেনি? টিক আছে৷ আসলে <> আমরা ওইরা গেছিগা৷ ...,Habiganj,0
1180,1181,rec_34.wav,rec_34_audio_18.wav,sabrinayesminreya@gmail.com,"ফ্যাচ লাগি যায়রেবো, ফ্যাচ লাগি যায়। <> আইচ্ছা ...",Habiganj,0
1181,1182,rec_34.wav,rec_34_audio_19.wav,sabrinayesminreya@gmail.com,"ইটা খেটা যাইতাছে? ওবা, আও না। দুরু মিয়া, আও। <>",Habiganj,0
1182,1183,rec_34.wav,rec_34_audio_20.wav,sabrinayesminreya@gmail.com,ইটা <> বালা না। না ইটা <> তে খালকু।,Habiganj,0


In [ ]:
habiganj_main.columns

Index(['Unnamed: 0', 'Original', 'External_ID', 'Annotator', 'Contents',
       'District', 'Time'],
      dtype='object')

In [ ]:
habiganj_main = habiganj_main.drop(labels = ['Unnamed: 0','Original','Annotator','District','Time'], axis = 1)
habiganj_main

,External_ID,Contents
0,rec_01_audio_0.wav,আইলাম তোমরার সাতে মাতবার লাইগা। বন্দ খর। সুফিয়...
1,rec_01_audio_1.wav,"না এরা দিতো না, এরা হুজুর মানুষ। কেরে দিতো না?..."
2,rec_01_audio_2.wav,<> ছেলে সামলানি লাগে। তে ফুলা কিতা অখনোই সামলা...
3,rec_01_audio_3.wav,"ইখানো তো মনো খর খাজ, অন্য খাজ তো খরন লাগে না, ..."
4,rec_01_audio_4.wav,"বাফের বাইত আইলে রিল্যাক্স, যেবালা ইচ্ছা ইবালা ..."
...,...,...
1179,rec_34_audio_17.wav,টিক আছেনি? টিক আছে৷ আসলে <> আমরা ওইরা গেছিগা৷ ...
1180,rec_34_audio_18.wav,"ফ্যাচ লাগি যায়রেবো, ফ্যাচ লাগি যায়। <> আইচ্ছা ..."
1181,rec_34_audio_19.wav,"ইটা খেটা যাইতাছে? ওবা, আও না। দুরু মিয়া, আও। <>"
1182,rec_34_audio_20.wav,ইটা <> বালা না। না ইটা <> তে খালকু।


In [ ]:
len(habiganj_main)

1184

In [ ]:
habiganj_b_whisper = pd.read_csv('/content/drive/MyDrive/Habiganj_Audio/Bangla_Whisper/habiganj-banglaWhisper-inferences.csv')
habiganj_b_whisper

,External_ID,contents
0,rec_01_audio_0,"কোগোয়ালাম, তুমরার সাথে মাঠবালে গ্যান। মতিরামা..."
1,rec_01_audio_1,"নাই, আরা দ্বিতা, না উজুর মানুষ। কে আরা যারেজনা..."
2,rec_01_audio_10,তুমিতে গানা তা-কোমি নেতে ওয়ার মেল ফেরে গেয়েত...
3,rec_01_audio_11,"এরা মানয় হরে বলো লুকমানু সাহিসে, না এরত মানয় হ..."
4,rec_01_audio_12,অগো উফলার নাম কিতার হুই তায়না তো আম। অগোলনাম ...
...,...,...
1179,rec_34_audio_6,"দুজন্মরূপে গেছে, দুজন্তুটা চাঁঠদোর্থ, একদন এপক..."
1180,rec_34_audio_7,"যিতে বনা করত ভাই যে আজকে দুই জনে লাগসে, আমরা স..."
1181,rec_34_audio_8,"আসফর, বিপদ্ন, মাংস হরব, সে ওই শিল্লাহ আল্লাহে ..."
1182,rec_34_audio_9,উইসবে যামার লাগব। হোন গেরামের যে ভাই ওই স্মৃতি...


In [ ]:
len(habiganj_b_whisper)

1184

In [ ]:
for i in habiganj_b_whisper['External_ID']:
    habiganj_b_whisper['External_ID'] = habiganj_b_whisper['External_ID'].replace([i], f"{i}.wav")


habiganj_b_whisper

,External_ID,contents
0,rec_01_audio_0.wav,"কোগোয়ালাম, তুমরার সাথে মাঠবালে গ্যান। মতিরামা..."
1,rec_01_audio_1.wav,"নাই, আরা দ্বিতা, না উজুর মানুষ। কে আরা যারেজনা..."
2,rec_01_audio_10.wav,তুমিতে গানা তা-কোমি নেতে ওয়ার মেল ফেরে গেয়েত...
3,rec_01_audio_11.wav,"এরা মানয় হরে বলো লুকমানু সাহিসে, না এরত মানয় হ..."
4,rec_01_audio_12.wav,অগো উফলার নাম কিতার হুই তায়না তো আম। অগোলনাম ...
...,...,...
1179,rec_34_audio_6.wav,"দুজন্মরূপে গেছে, দুজন্তুটা চাঁঠদোর্থ, একদন এপক..."
1180,rec_34_audio_7.wav,"যিতে বনা করত ভাই যে আজকে দুই জনে লাগসে, আমরা স..."
1181,rec_34_audio_8.wav,"আসফর, বিপদ্ন, মাংস হরব, সে ওই শিল্লাহ আল্লাহে ..."
1182,rec_34_audio_9.wav,উইসবে যামার লাগব। হোন গেরামের যে ভাই ওই স্মৃতি...


In [ ]:
len(habiganj_b_whisper)

1184

In [ ]:
def make_equal(dataset):

    df1 = habiganj_main
    df2 = dataset

    df = df1.join(df2.set_index('External_ID'), on='External_ID', how = 'left')
    df = df.drop(columns=["Contents"], axis = 1)
    df['contents'] = df['contents'].fillna("Null")
    return df


def errors(dataset, dname):

    dName = str(dataset)

    if len(dataset) == len(habiganj_main):

        CER = []
        WER = []

        dataset['contents'] = dataset['contents'].replace(["<>"], "")

        k = 1
        for i, j in zip(habiganj_main['Contents'],dataset['contents']):
            # print("{} -->", k)
            # print("main: {}",i)
            # print("\n")
            # print("inference: {}",j)
            # print("**************************\n")
            WER.append(wer(str(i), str(j)))
            k += 1


        for i, j in zip(habiganj_main['Contents'],dataset['contents']):
            CER.append(cer(str(i), str(j)))


        habiganj_main[f"CER ({dname})"] = CER
        habiganj_main[f"WER ({dname})"] = WER

    else:
        print("In\n")
        dataset = make_equal(dataset)
        errors(dataset,dname)



    return habiganj_main

In [ ]:
df = errors(habiganj_b_whisper, 'B_Whisper')
df

,External_ID,Contents,CER (B_Whisper),WER (B_Whisper)
0,rec_01_audio_0.wav,আইলাম তোমরার সাতে মাতবার লাইগা। বন্দ খর। সুফিয়...,0.671569,0.945946
1,rec_01_audio_1.wav,"না এরা দিতো না, এরা হুজুর মানুষ। কেরে দিতো না?...",0.619048,0.906977
2,rec_01_audio_2.wav,<> ছেলে সামলানি লাগে। তে ফুলা কিতা অখনোই সামলা...,0.748918,1.000000
3,rec_01_audio_3.wav,"ইখানো তো মনো খর খাজ, অন্য খাজ তো খরন লাগে না, ...",0.814815,0.979167
4,rec_01_audio_4.wav,"বাফের বাইত আইলে রিল্যাক্স, যেবালা ইচ্ছা ইবালা ...",0.761905,1.000000
...,...,...,...,...
1179,rec_34_audio_17.wav,টিক আছেনি? টিক আছে৷ আসলে <> আমরা ওইরা গেছিগা৷ ...,0.991736,1.000000
1180,rec_34_audio_18.wav,"ফ্যাচ লাগি যায়রেবো, ফ্যাচ লাগি যায়। <> আইচ্ছা ...",2.263158,3.000000
1181,rec_34_audio_19.wav,"ইটা খেটা যাইতাছে? ওবা, আও না। দুরু মিয়া, আও। <>",3.468085,2.500000
1182,rec_34_audio_20.wav,ইটা <> বালা না। না ইটা <> তে খালকু।,2.514286,2.000000


In [ ]:
metrics = {}


Avg_CER_b_whis = np.mean(df["CER (B_Whisper)"])
Avg_WER_b_whis = np.mean(df["WER (B_Whisper)"])

metrics["B_Whisper"] = f"Avg_WER : {Avg_WER_b_whis} | Avg_CER : {Avg_CER_b_whis}"

metrics

{'B_Whisper': 'Avg_WER : 1.2379977705543284 | Avg_CER : 1.2697942310655022'}